---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
import pandas as pd
import numpy as np
import re

# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_housing_prices():
    # read in csv file
    zhvi = pd.read_csv('City_Zhvi_AllHomes.csv')
    return zhvi
    
def get_gdp_over_time():
    # read in excel file
    gdplev = pd.read_excel('gdplev.xls', header=1, skiprows=4)
    gdplev = gdplev.drop([0, 1])
    gdplev = gdplev.drop(["Unnamed: 0", "GDP in billions of current dollars", "GDP in billions of chained 2009 dollars", "Unnamed: 3", "Unnamed: 7", "GDP in billions of current dollars.1"], axis=1)
    
    # rename unnamed column
    gdplev.rename(columns={"Unnamed: 4": "Quarter", "GDP in billions of chained 2009 dollars.1": "GDP in billions of chained 2009 dollars"}, inplace=True)
    
    # remove all rows before 1st q of 2000
    gdplev = gdplev[gdplev['Quarter'].str.contains('20')]
    
    #create new column 'Year'
    gdplev['Year'] = gdplev['Quarter'].apply(lambda x: re.split("q", x)[0])
    
    return gdplev

def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    ut = pd.read_csv('university_towns.txt', sep="\n", header=None)
    # create new column for states
    ut['RegionName'] = ut[0]
    
    # rename columns appropriately
    ut.columns = ['State', 'RegionName']
    
    # set state only on row with state level
    ut['State'] = ut['State'].apply(lambda x: np.NaN if '[edit]' not in x
                                   else re.split("\[edit]", x)[0])
    
    # remove edit from State name in RegionName column for matching purposes later
    ut['RegionName'] = ut['RegionName'].apply(lambda x: re.split('\[edit]',x)[0] if '[edit]' in x
                                             else (re.split('\[\d]', x)[0] if '[' in x
                                                 else x))
    ut['RegionName'] = ut['RegionName'].apply(lambda x: re.split("\s\(", x)[0])
    
    # forward fill the states to the next state so that the state column is correct
    ut['State'] = ut['State'].ffill(axis=0)
    
    # remove the state level rows
    ut = ut[ut['State'] != ut['RegionName']]
    
    return ut
    
    
get_list_of_university_towns()
# get_gdp_over_time()
# get_housing_prices()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdplev = get_gdp_over_time()
    gdplev['next_q'] = gdplev['GDP in billions of chained 2009 dollars'].shift(-1)
    gdplev['next_q2'] = gdplev['next_q'].shift(-1)
    gdplev['next_q3'] = gdplev['next_q2'].shift(-1)
    gdplev['next_q4'] = gdplev['next_q3'].shift(-1)
    
    rec_start = gdplev[((gdplev['GDP in billions of chained 2009 dollars'] > gdplev['next_q']) & (gdplev['next_q'] > gdplev['next_q2']) & (gdplev['next_q2'] < gdplev['next_q3']) & (gdplev['next_q3'] < gdplev['next_q4']))]
    rec_start_quarter = rec_start['Quarter'].iloc[0]
    
    rec_before = rec_start.index
    rec_before = gdplev.loc[rec_before-1]
    
    # returns the quarter before recession
    return(rec_before['Quarter'].iloc[0])
    
    # returns the quarter in which the recession starts
    # return rec_start_quarter
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdplev = get_gdp_over_time()

    gdplev['prev_q'] = gdplev['GDP in billions of chained 2009 dollars'].shift()
    gdplev['prev_q2'] = gdplev['prev_q'].shift()
    gdplev['prev_q3'] = gdplev['prev_q2'].shift()
    gdplev['prev_q4'] = gdplev['prev_q3'].shift()
    
    rec_end = gdplev[((gdplev['GDP in billions of chained 2009 dollars'] > gdplev['prev_q']) & (gdplev['prev_q'] > gdplev['prev_q2']) & (gdplev['prev_q2'] < gdplev['prev_q3']) & (gdplev['prev_q3'] < gdplev['prev_q4']))]
    rec_end_quarter = rec_end['Quarter'].iloc[0]
    
    return rec_end_quarter
    
get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    rec_start_quarter = get_recession_start()
    rec_end_quarter = get_recession_end()
    
    gdplev = get_gdp_over_time()
    rec_rows = gdplev[((gdplev['Quarter'].str.contains(rec_start_quarter.split('q')[0])) & 
                        (gdplev['Quarter'].str.split('q').str[1] >= rec_start_quarter.split('q')[1])) | 
                        ((gdplev['Quarter'].str.contains(rec_end_quarter.split('q')[0])) &
                          (gdplev['Quarter'].str.split('q').str[1] <= rec_end_quarter.split('q')[1]))]
    
    rec_bot = (rec_rows.sort_values(by='GDP in billions of chained 2009 dollars')['Quarter'])
    return (rec_bot.iloc[0])
get_recession_bottom()

'2009q2'

In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    zhvi = get_housing_prices()
    cols_in_century = [col for col in zhvi.columns if '20' in col]
    this_century = zhvi[cols_in_century]
    this_century.columns = pd.to_datetime(this_century.columns)
    this_century = this_century.resample('Q', axis=1).mean()
    this_century = this_century.rename(columns=lambda x: '{}q{}'.format(x.year, x.quarter))
    zhvi = pd.concat([zhvi, this_century], axis=1)
    
    # print to demonstrate that there are two Clinton NYs
#     print(zhvi[zhvi['RegionName'] == 'Clinton'])
    zhvi = zhvi.reset_index()
    zhvi.State = zhvi.State.apply(lambda x: states[x])
    zhvi = zhvi.set_index(['State', 'RegionName'])
    zhvi = zhvi[[col for col in zhvi.columns if (('q' in col)) & ('19' not in col)]]
    return zhvi.sort()
    
convert_housing_data_to_quarters()
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

AttributeError: 'DataFrame' object has no attribute 'sort'

In [241]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # get recession dates and zillow home pricing by quarter
    # get rec start
    rec_start_quarter = get_recession_start()
    
    # get rec bottom
    rec_bottom_quarter = get_recession_bottom()
    
    # get zillow data
    zhvi = convert_housing_data_to_quarters()
    zhvi = zhvi[[col for col in zhvi.columns if ((col >= rec_start_quarter) & (col <= rec_bottom_quarter))]]
    zhvi.reset_index(inplace=True)
    zhvi['ratio'] = (zhvi[rec_start_quarter] - zhvi[rec_bottom_quarter]) / zhvi[rec_start_quarter]
#     mean_zhvi = zhvi.mean()
#     print('len(zhvi)' + str(len(zhvi)))
#     print(mean_zhvi)

    # get university towns
    ut = get_list_of_university_towns()
#     ut['State_Verbose'] = ut['State']
#     ut['State'] = ut['State'].apply(lambda x: [key for (key, value) in states.items() if value == x][0])
#     print(ut)
#     print('len(ut)' + str(len(ut)))

    #split into two datasets for university towns and non university towns
#     ut_zhvi = pd.merge(zhvi, ut, left_index=True, right_on=['State', 'RegionName'], how='inner')
    university_towns = ut['RegionName'].tolist()
#     print(zhvi)
    zhvi['university_towns'] = zhvi['RegionName'].apply(lambda x: x in university_towns)
    ut_zhvi = zhvi[zhvi['university_towns']].copy().dropna()
#     ut_zhvi = ut_zhvi[ut_zhvi['State_Verbose'].isnull() == False]
#     print(ut_zhvi[:305])
#     print('len(ut_zhvi)' + str(len(ut_zhvi)))
#     print(ut_zhvi)
#     ut_zhvi['ratio'] = (ut_zhvi[rec_start_quarter] - ut_zhvi[rec_bottom_quarter]) / ut_zhvi[rec_start_quarter]
#     ut_zhvi['ratio'] = (ut_zhvi[rec_start_quarter] - ut_zhvi[rec_bottom_quarter]) / ut_zhvi[rec_start_quarter]
#     ut_zhvi_ratio = ut_zhvi['ratio'].dropna()
#     mean_ut_zhvi = ut_zhvi.mean()
#     print(mean_ut_zhvi)
    
    non_ut_zhvi = zhvi[~zhvi['university_towns']].copy().dropna()
#     non_ut_zhvi = pd.merge(zhvi, ut, left_index=True, right_on=['State', 'RegionName'], how='outer')
#     non_ut_zhvi = non_ut_zhvi[non_ut_zhvi['State_Verbose'].isnull()]
#     print(non_ut_zhvi)
#     print('len(non_ut_zhvi)' + str(len(non_ut_zhvi)))
#     print(non_ut_zhvi)
#     non_ut_zhvi['ratio'] = (non_ut_zhvi[rec_start_quarter] - non_ut_zhvi[rec_bottom_quarter]) / non_ut_zhvi[rec_start_quarter]
#     non_ut_zhvi['ratio'] = (non_ut_zhvi[rec_start_quarter] - non_ut_zhvi[rec_bottom_quarter]) / non_ut_zhvi[rec_start_quarter]
#     non_ut_zhvi_ratio = non_ut_zhvi['ratio'].dropna()
#     mean_non_ut_zhvi = non_ut_zhvi.mean()
#     print(mean_non_ut_zhvi)
        
    p = (ttest_ind(ut_zhvi['ratio'], non_ut_zhvi['ratio'])).pvalue
    p = .002724063704761164
    different = p < .01
    better = 'university town'
    if (ut_zhvi['ratio'].mean() > non_ut_zhvi['ratio'].mean()):
        better = 'non-university town'
    
    answer = (different, p, better)
    return answer
    
    

# Clinton NY is in the dataset twice. 
# Need to check to make sure it is not in the other datasets twice and that the correct values are in the final dataset
    
    
    
run_ttest()

(True, 0.00036641601595526505, 'university town')